In [1]:
# 1. Library imports
import pandas as pd
from IPython.display import Markdown, display
from extraction import llmAgent
from ppi_deprescribe import merge_results, ppi_deprescribe
import os 
from pathlib import Path 


/Users/reedevans/Documents/mids/deprescribe_main/DataSci210_MedicationDeprescriber/src/extraction.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
from dotenv import load_dotenv  # Import load_dotenv

load_dotenv() 

True

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [4]:
import json
from pathlib import Path

from botocore.exceptions import ClientError

from langchain_aws import ChatBedrock

from query import DataLoader

#model = 'meta.llama3-70b-instruct-v1:0'
model = 'us.anthropic.claude-3-5-haiku-20241022-v1:0'
#self.llm = ChatGroq(temperature=0, model=llama_31, api_key=groq_key)

llm = ChatBedrock(
    model_id=model,
    model_kwargs=dict(temperature=0),
    # other params...
)

In [10]:

path = Path('../Data')
groq = os.environ['groqkey']
pd.set_option('display.max_colwidth', None)
dl = DataLoader(path)

In [6]:
enc_key = 'D8017C77BA15FA'

,review_by,validated_by,key,reason,recommendation,diagnosis,GS_response,Note
0,Ben,NaN,D6253A5CE371EA,example with gastrointestinal bleed should continue PPI,continue,3-Upper GI Bleed,"This patient has a history of a gastrointestinal bleeding which was present on the admission diagnosis. The pantoprazole 20mg taken twice daily should be continued. The GI bleed is mentioned in the note from 09/14/2023, on 09/17/2023, and on 09/20/2023.",NaN
1,Ben,NaN,DFAFED1811B871,example with a documented history of a GI bleed should continue PPI,continue,3-Upper GI Bleed,This patient has a history of a gastrointestinal bleed in the present on admit diagnosis. The lansoprazole 30mg daily should be continued. Note from 09/02/2023 and 09/03/2023 mention the GI bleed.,NaN
2,Ben,NaN,D4865B8BBB294E,example with GI bleeding that occurred during inpatient stay should continue PPI,continue,3-Upper GI Bleed,GI bleeding developed during hospital stay and is present in the hospital acquired diagnosis. The patient should continue the omeprazole 20mg taking once daily. 09/13/2023 note mentions bleeding ulcer.,NaN
3,Ben,NaN,D00F3A8D5F43B2,example with symptoms of heartburn (upper GI symptoms) evaluate for endoscopy to determine if PPI is needed,deprescribe,9-Treated GERD,09/20/2023 inpatient nursing note mentions use of pantoprazole for heartburn. Attempt deprescribing of lansoprazole 30mg at discharge.,NaN
4,Ben,NaN,D8017C77BA15FA,"14d h pylori treatment if pt treated for 2wks and asymptomatic, stop PPI",stop,7-Treated H Pylori,Stop pantoprazole 40mg when H. Pylori treatment scheduled from 07/24-08/07 is complete. Documented in note from Anesthesiology on 09/01/2023,NaN


In [27]:
diagnoses = {"diagnoses": [
        "Barretts Esophagus or esophageal cell changes",
        "Chronic NSAID used with bleeding risk or GI prophylaxis NSAID use",
        "Severe esophagitis including bleeding esophagitis or esophageal ulcer",
        "History of bleeding GI/gastric ulcer, upper GI bleed, or peptic ulcer hemorrhage",
        "Peptic Ulcer Disease or Gastroduodenal ulcer treated for 2 - 12 weeks caused from H Pylori infection or NSAID use without bleeding",
        "Upper GI Symptoms such as reflux, difficulty swallowing, nausea, or vomiting without endoscopy - asymptomatic for 3 consecutive days",
        "ICU Stress Ulcer Prophylaxis",
        "Completed Heliobacter Pylori (H. Pylori) infection treated for 14 days with combination therapy",
        "Mild to moderate esophagitis or esophageal inflammation",
        "Treated Gastroesophageal Reflux Disease (GERD) or reflux symptoms such as acid reflux, heartburn, or regurgitation",
    ]
    }

In [60]:
labeled_data = pd.read_csv('../Data/LabeledResponses.csv')
#labeled_data = labeled_data.head(1)
labeled_data.head()


,review_by,validated_by,key,reason,recommendation,diagnosis,GS_response,Note
0,Ben,NaN,D6253A5CE371EA,example with gastrointestinal bleed should continue PPI,continue,3-Upper GI Bleed,"This patient has a history of a gastrointestinal bleeding which was present on the admission diagnosis. The pantoprazole 20mg taken twice daily should be continued. The GI bleed is mentioned in the note from 09/14/2023, on 09/17/2023, and on 09/20/2023.",NaN
1,Ben,NaN,DFAFED1811B871,example with a documented history of a GI bleed should continue PPI,continue,3-Upper GI Bleed,This patient has a history of a gastrointestinal bleed in the present on admit diagnosis. The lansoprazole 30mg daily should be continued. Note from 09/02/2023 and 09/03/2023 mention the GI bleed.,NaN
2,Ben,NaN,D4865B8BBB294E,example with GI bleeding that occurred during inpatient stay should continue PPI,continue,3-Upper GI Bleed,GI bleeding developed during hospital stay and is present in the hospital acquired diagnosis. The patient should continue the omeprazole 20mg taking once daily. 09/13/2023 note mentions bleeding ulcer.,NaN
3,Ben,NaN,D00F3A8D5F43B2,example with symptoms of heartburn (upper GI symptoms) evaluate for endoscopy to determine if PPI is needed,deprescribe,9-Treated GERD,09/20/2023 inpatient nursing note mentions use of pantoprazole for heartburn. Attempt deprescribing of lansoprazole 30mg at discharge.,NaN
4,Ben,NaN,D8017C77BA15FA,"14d h pylori treatment if pt treated for 2wks and asymptomatic, stop PPI",stop,7-Treated H Pylori,Stop pantoprazole 40mg when H. Pylori treatment scheduled from 07/24-08/07 is complete. Documented in note from Anesthesiology on 09/01/2023,NaN


In [75]:

df = []

count = 0
for index, encounter in labeled_data.iterrows():
    count += 1
    print(count)
    notes = dl.get_notes_data(encounter.key)
    if notes is not None:
        for index, row in notes.iterrows():
            note_text = row['NoteText'] 
            prompt = f""" Based on the following clinician note <note> {note_text} </note> look for any diagnoses related to this list and provide a short summary two or three sentences only. 
                Here are the diagnoses we are looking for {str(diagnoses)}. Always start your response with a "yes" or "no" based on finding any of the listed diagnoses. 
                    """
            llm_response = llm.invoke(prompt)
            response = str(llm_response.content)

            print(response[:100])

            if "yes" in response.lower():
                notes.at[index, 'llm_summary'] = response
                print('**** yes *****')
            else:
                notes.at[index, 'llm_summary'] = 'No diagnoses found'
        df.append(notes)


updated_notes = pd.concat(df)

notes['NoteText'] = notes.apply(lambda row: f"{row['llm_summary']} {row['NoteText']}" if row['llm_summary'] != "No diagnoses found" else row['NoteText'], axis=1)

updated_notes.to_csv('updated_labeled_notes.csv', index=False)


1
Yes, based on the note, there are indications of an acute GI bleed with bloody stool requiring multi
**** yes *****
No. Based on the provided chest X-ray clinician note, there is insufficient information to determine
Yes, based on the note, here's a summary:

The patient has a history of mild gastritis with small AV
**** yes *****
No. Based on the provided clinician note, there is no clear indication of any of the listed diagnose
Yes, the note indicates a GI Bleed diagnosis, which is related to several of the listed diagnoses, p
**** yes *****
No. 

The provided ultrasound note appears to be a bilateral lower extremity venous Doppler study do
After carefully reviewing the clinician note, I cannot definitively identify any of the specific dia
After carefully reviewing the clinician note, here's my assessment:

No. 

While the note mentions s
Let me carefully review the note for the listed diagnoses:

Yes, there are some relevant findings:


**** yes *****
After carefully reviewing the

In [77]:
new_notes = pd.read_csv('updated_labeled_notes.csv')

In [80]:
len(new_notes.EncounterKey.unique())

42

In [49]:
notes.to_csv("notes_summary_out.csv",index=False)

In [68]:
dl.get_notes_data("D8017C77BA15FA")

In [33]:
# llm.invoke(f""" Based on the following clinician note <note> {note} </note> look for any diagnoses that could be related to this patients proton pump inhibitor (PPI) prescription. We know they are prescribed a PPI. We want to know why" 
#             """)

AIMessage(content='Based on the clinical note, there are a few potential diagnoses related to the patient\'s PPI prescription:\n\n1. H Pylori Infection:\nThe note specifically mentions "H Pylori infection (treatment day 10)" which is a common indication for PPI prescription. PPIs are typically used in H. Pylori treatment protocols, often as part of a triple therapy regimen to reduce stomach acid and help eradicate the bacteria.\n\n2. Potential Underlying Conditions:\nWhile not explicitly stated in this note, PPIs are commonly prescribed for:\n- Gastroesophageal reflux disease (GERD)\n- Peptic ulcer disease\n- Gastritis\n\nIn this specific case, the note indicates:\n- Abdominal pain\n- Acute gastroenteritis\n- Altered mental status\n\nThese symptoms could be related to gastrointestinal issues that might have initially prompted the PPI prescription.\n\nThe note doesn\'t provide complete details about the original reason for the PPI prescription, but the H. Pylori treatment is the most cl